# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

Justin Frauenhofer (jtfrauen@ucsc.edu)

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [ ]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

In [ ]:
def count_potential_elimations(b: chess.Board, player: bool):
    """
    This function counts the number of potential piece eliminations for the given player.
    """
    current_turn = b.turn

    b.turn = player
    count = 0
    for move in b.legal_moves:
        if b.is_capture(move):
            count += 1

    b.turn = current_turn
    return count

def count_pieces(b: chess.Board, player: bool):
    """
    This function counts the number of pieces for the given player.
    """
    
    count = 0
    for square in chess.SQUARES:
        piece = b.piece_at(square)
        if piece is not None and piece.color == player:
            count += 1
    return count

def has_opponent_in_check(b: chess.Board, player: bool):
    """
    Returns 0 if player is in check, 1 if nobody is in check, or 2 if player has them in check.
    """

    current_turn = b.turn
    result = 1 # Default to nobody in check

    # If we're in check, return 0
    b.turn = player
    if b.is_check():
        result = 0

    # If they're in check, return 2
    b.turn = not player
    if b.is_check():
        result = 2

    b.turn = current_turn
    return result

def has_opponent_in_check_mate(b: chess.Board, player: bool):
    """
    Returns 0 if player is in checkmate, 1 if nobody is in checkmate, or 2 if player has them in checkmate.
    """

    current_turn = b.turn
    result = 1 # Default to nobody in checkmate

    # If we're in checkmate, return 0
    b.turn = player
    if b.is_checkmate():
        result = 0

    # If they're in checkmate, return 2
    b.turn = not player
    if b.is_checkmate():
        result = 2

    b.turn = current_turn
    return result


def evaluation(b: chess.Board, player: bool):
    """
    This function evaluates a board position and returns a score value. It uses the following heuristics:
     
    1.) Number of potential piece eliminations
    2.) Difference in quantity of pieces
    3.) King is in check
    4.) King is in checkmate (goal state)
    
    All evaluations are weigthed appropriately.
    
    Parameters:
    - board: the chess board that the knight is moving upon   // What does the knight have to do with anything?
    - player: the colour of the active player (True -> white, False -> black)
    
    Returns:
    - an integer score for a board state, higher values are better for the given player.
    """

    potential_elims = count_potential_elimations(b, player)
    piece_diff = count_pieces(b, player) - count_pieces(b, not player)
    check = has_opponent_in_check(b, player)
    check_mate = has_opponent_in_check_mate(b, player)

    # Arbitrary weights based on what tickles my fancy
    score = 0
    score += 1 * piece_diff
    score += 2 * potential_elims
    score += 4 * (check / 2) # Normalize check heuristic
    
    # Goal state check
    if check_mate == 0:
        # We're in checkmate
        score = -inf
    elif check_mate == 2:
        # We've got them!
        score = inf

    return score

Unit tests

In [ ]:
# Unit test for count potential elims
b = chess.Board("8/8/8/8/8/8/8/8")

# Set black king to be in check front white queen
b.set_piece_at(chess.H8, chess.Piece.from_symbol('k'))
b.set_piece_at(chess.H1, chess.Piece.from_symbol('Q'))

assert count_potential_elimations(b, chess.WHITE) == 1
assert count_potential_elimations(b, chess.BLACK) == 0

b.set_piece_at(chess.H1, None)
b.set_piece_at(chess.G1, chess.Piece.from_symbol('Q'))

assert count_potential_elimations(b, chess.WHITE) == 0
assert count_potential_elimations(b, chess.BLACK) == 0

b.set_piece_at(chess.G1, None)
b.set_piece_at(chess.G8, chess.Piece.from_symbol('Q'))

assert count_potential_elimations(b, chess.WHITE) == 1
assert count_potential_elimations(b, chess.BLACK) == 1

In [ ]:
# Unit tests for count pieces

b = chess.Board("8/8/8/8/8/8/8/8")
b.set_piece_at(chess.H1, chess.Piece.from_symbol('K'))
b.set_piece_at(chess.H2, chess.Piece.from_symbol('Q'))

assert count_pieces(b, chess.WHITE) == 2
assert count_pieces(b, chess.BLACK) == 0

b.set_piece_at(chess.H3, chess.Piece.from_symbol('q'))

assert count_pieces(b, chess.WHITE) == 2
assert count_pieces(b, chess.BLACK) == 1


In [ ]:
# Unit test for check heuristic

# Test board
b = chess.Board("8/8/8/8/8/8/8/8")

# Set black king to be in check front white queen
b.set_piece_at(chess.H8, chess.Piece.from_symbol('k'))
b.set_piece_at(chess.H1, chess.Piece.from_symbol('Q'))

# If black is our max player, and black is in check, then return 0
assert has_opponent_in_check(b, chess.BLACK) == 0

# If white is our max player, and black is in check, then return 2
assert has_opponent_in_check(b, chess.WHITE) == 2

# Move queen away from king
b.set_piece_at(chess.H1, None)
b.set_piece_at(chess.G1, chess.Piece.from_symbol('Q'))

# No one is in check
assert has_opponent_in_check(b, chess.WHITE) == 1
assert has_opponent_in_check(b, chess.BLACK) == 1

In [ ]:
# Unit tests for checkmate heuristic

# Test board
b = chess.Board("8/8/8/8/8/8/8/8")
b.set_piece_at(chess.H8, chess.Piece.from_symbol('k'))
b.set_piece_at(chess.G8, chess.Piece.from_symbol('Q'))
b.set_piece_at(chess.F8, chess.Piece.from_symbol('R'))

assert has_opponent_in_check_mate(b, chess.WHITE) == 2
assert has_opponent_in_check_mate(b, chess.BLACK) == 0

b.set_piece_at(chess.G8, None)
b.set_piece_at(chess.G1, chess.Piece.from_symbol('Q'))

assert has_opponent_in_check_mate(b, chess.WHITE) == 1
assert has_opponent_in_check_mate(b, chess.BLACK) == 1


In [ ]:
# Unit test for evaluation function

b = chess.Board("8/8/8/8/8/8/8/8")
b.set_piece_at(chess.H8, chess.Piece.from_symbol('k'))
b.set_piece_at(chess.G8, chess.Piece.from_symbol('Q'))
b.set_piece_at(chess.F8, chess.Piece.from_symbol('R'))

# assert evaluation(b, chess.WHITE) == inf
# assert evaluation(b, chess.BLACK) == -inf

b.set_piece_at(chess.G8, None)
b.set_piece_at(chess.F8, None)

# White has no pieces, black has one piece
# assert evaluation(b, chess.WHITE) < evaluation(b, chess.BLACK)

# White has one piece, black has one piece, black is in check
b.set_piece_at(chess.H1, chess.Piece.from_symbol('Q'))
# assert evaluation(b, chess.WHITE) > evaluation(b, chess.BLACK)

# White has one piece, black has two pieces, but white has an elimination
b.set_piece_at(chess.H5, chess.Piece.from_symbol('p'))
assert evaluation(b, chess.WHITE) >= evaluation(b, chess.BLACK)

## Q1

Implement the minimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [ ]:
def get_minimax_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """

    def helper(local_b, local_player, local_depth, alpha, beta):
        # YOUR CODE HERE
        raise NotImplementedError
    
    # YOUR CODE HERE
    raise NotImplementedError

## Q2

Implement the expectimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [ ]:
def get_expectimax_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """

    def helper(local_b, local_player, local_depth, alpha, beta):
        # YOUR CODE HERE
        raise NotImplementedError
    
    # YOUR CODE HERE
    raise NotImplementedError

## Q3

Plays against the strongest grading agent, so copy your best agent here.

In [ ]:
def get_best_move(b: chess.Board, player: bool, depth: int):   
    """
    This function chooses the best move for the given board position, player, and depth.
    
    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.
    
    Returns:
    A single chess.Move type object.
    """    
    
    # YOUR CODE HERE
    raise NotImplementedError

## Local Testing
For your convenience, here are some helper functions to run games and allow you to test your agents.

In [ ]:
MAX_DEPTH = 3
SEED = random.Random(0)  # Feel free to set/reset the seed for testing purposes!

def get_random_move(b:chess.Board, *_):
    return SEED.choice(list(b.legal_moves))

def test_game(p1, p2):
    """
    A function that plays a chess game with visuals
    
    Parameters:
    - p1: the first chess agent, playing as the white player
    - p2: the second chess agent, playing as the black player
    
    Returns:
    Nothing, just plays the game one move at a time.
    Press enter in the popup box to play the next move.
    Type 'q' into the popup box and press enter to stop the game (you can also interrupt the kernel if this fails.)
    """
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate()  and not board.is_stalemate() and not board.can_claim_draw():
        clear_output(True)
        
        move = current_player(board, board.turn, MAX_DEPTH)
        board.push(move)
        display(board)

        current_player = p1 if board.turn else p2

        if input() == "q":
            break
    

In [ ]:
# Random Vs. Random
random_agent = get_random_move
test_game(random_agent, random_agent)

In [ ]:
# Minimax Vs. Random
random_agent = get_random_move
minimax_agent = get_minimax_move
test_game(minimax_agent, random_agent)


In [ ]:
# Expectimax Vs. Random
random_agent = get_random_move
expectimax_agent = get_expectimax_move
test_game(expectimax_agent, random_agent)

In [ ]:
# Run several games at once without graphics
def grade_game(p1, p2):
    """
    Grades a single game at a time.
    (Note: differs from actual grading script, which has some ways to resolve draws.)

    Return legend:
    0: error during game
    1: p1 wins through checkmate
    2: p2 wins through checkmate
    3: draw
    """
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate() and not board.is_stalemate() and not board.can_claim_draw():
        move = current_player(board, board.turn, MAX_DEPTH)
        try:
            board.push(move)
        except:            
            print(f"Error while grading game, move = {move}, current_player = {current_player}")
            print(board)
            return -1

        current_player = p1 if board.turn else p2
    
    outcome = board.outcome()
    if outcome is not None:
        if outcome.winner:
            return 1
        else:
            return 2
    
    return 3

In [ ]:
# Play many games in a row without visuals (faster)
random_agent = get_random_move
best_agent = get_best_move
num_games = 5

print(f"Playing {num_games} games...")
for i in range(1, num_games+1):
    result = grade_game(best_agent, random_agent)

    if result == 0:
        print(f"Game {i}: error pushing a move during gameplay")
        
    elif result == 1:
        print(f"Game {i}: p1 wins")
        
    elif result == 2:
        print(f"Game {i}: p1 loses")
    
    elif result == 3:
        print(f"Game {i}: draw")